In [1]:
import sys
import numpy as np
import pandas as pd
import scipy
import copy
import random
import math
from scipy import stats
from scipy.stats import rankdata
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn import metrics, preprocessing
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor, plot_tree
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import Markdown, display
np.random.seed(1)

In [2]:
cols = ['age', 'workclass', 'fnlwgt', 'education', 'education.num', 'marital', 'occupation', 'relationship', 'race', 'gender', 'capgain', 'caploss', 'hours', 'country', 'income']
df_train = pd.read_csv('adult.data', names=cols, sep=",")
df_test = pd.read_csv('adult.test', names=cols, sep=",")

**One-hot encoding**

In [3]:
 def one_hot_encode(df):
    df.isin(['?']).sum(axis=0)

    # replace missing values (?) to nan and then drop the columns
    df['country'] = df['country'].replace('?',np.nan)
    df['workclass'] = df['workclass'].replace('?',np.nan)
    df['occupation'] = df['occupation'].replace('?',np.nan)

    # dropping the NaN rows now
    df.dropna(how='any',inplace=True)
    df['income'] = df['income'].map({'<=50K': 0, '>50K': 1}).astype(int)
    df = pd.concat([df, pd.get_dummies(df['gender'], prefix='gender')],axis=1)
    df = pd.concat([df, pd.get_dummies(df['race'], prefix='race')],axis=1)
    df = pd.concat([df, pd.get_dummies(df['marital'], prefix='marital')],axis=1)
    df = pd.concat([df, pd.get_dummies(df['workclass'], prefix='workclass')],axis=1)
    df = pd.concat([df, pd.get_dummies(df['relationship'], prefix='relationship')],axis=1)
    df = pd.concat([df, pd.get_dummies(df['occupation'], prefix='occupation')],axis=1)

    df = df.drop(columns=['workclass', 'gender', 'fnlwgt', 'education', 'occupation', \
                      'relationship', 'marital', 'race', 'country', 'capgain', \
                      'caploss'])
    return df

# one-hot encoding (for regression mdoels)
df_train = one_hot_encode(df_train)
df_test = one_hot_encode(df_test)

df_train = df_train.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

**Protected, privileged**

In [80]:
# protected: 'gender_Female'=1
# privileged: 'gender_Male'=1

**Parametric Model**

In [81]:
X_train = df_train.drop(columns='income')
y_train = df_train['income']

X_test = df_test.drop(columns='income')
y_test = df_test['income']

# size=500
# X_train = X_train[0:size]
# y_train = y_train[0:size]

X_train_orig = copy.deepcopy(X_train)
X_test_orig = copy.deepcopy(X_test)

# Scale data: regularization penalty default: ‘l2’, ‘lbfgs’ solvers support only l2 penalties. 
# Regularization makes the predictor dependent on the scale of the features.
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

clf = LogisticRegression(random_state=0, max_iter=300)

**Compute statistical parity difference**

In [82]:
def computeFairness(y_pred, X_test): 
    protected_idx = X_test[X_test['gender_Female']==1].index
    numProtected = len(protected_idx)
    privileged_idx = X_test[X_test['gender_Male']==1].index
    numPrivileged = len(privileged_idx)
    
    p_protected = 0
    for i in range(len(protected_idx)):
        p_protected += y_pred[protected_idx[i]][1]
    p_protected /= len(protected_idx)
    
    p_privileged = 0
    for i in range(len(privileged_idx)):
        p_privileged += y_pred[privileged_idx[i]][1]
    p_privileged /= len(privileged_idx)
    
    spd = p_protected - p_privileged
    return spd

**Influence of points computed using ground truth**

In [83]:
def ground_truth_influence(X_train, y_train, X_test, X_test_orig):
    clf.fit(X_train, y_train)
    y_pred = clf.predict_proba(X_test)
    spd_0 = computeFairness(y_pred, X_test_orig)

    delta_spd = []
    for i in range(len(X_train)):
        X_removed = np.delete(X_train, i, 0)
        y_removed = y_train.drop(index=i, inplace=False)
        clf.fit(X_removed, y_removed)
        y_pred = clf.predict_proba(X_test)
        delta_spd_i = computeFairness(y_pred, X_test_orig) - spd_0
        delta_spd.append(delta_spd_i)
    
    return delta_spd

**Loss function** (Log loss for logistic regression)

In [84]:
def logistic_loss(y_true, y_pred):
    loss = 0
    for i in range(len(y_true)):
        if (y_pred[i][1] != 0 and y_pred[i][0] != 0):
            loss += - y_true[i] * math.log(y_pred[i][1]) - (1 - y_true[i]) * math.log(y_pred[i][0])
    loss /= len(y_true)
    return loss

**Compute Accuracy** 

In [85]:
def computeAccuracy(y_true, y_pred):
    accuracy = 0
    for i in range(len(y_true)):
        idx = y_true[i]
        accuracy += y_pred[i][idx]
    accuracy /= len(y_true)
    return accuracy

**First-order derivative of loss function at z with respect to model parameters**

In [86]:
def del_L_del_theta_i(num_params, y_true, x, y_pred):
#     del_L_del_theta = np.ones((num_params, 1)) * ((1 - y_true) * y_pred[1] - y_true * y_pred[0])
    del_L_del_theta = np.ones((num_params, 1)) * (- y_true + y_pred[1])
    for j in range(1, num_params):
            del_L_del_theta[j] *=  x[j-1]
    return del_L_del_theta

**Hessian: Second-order partial derivative of loss function with respect to model parameters**

In [87]:
def hessian_one_point(num_params, x, y_pred):
    H = np.ones((num_params, num_params)) * (y_pred[0] * y_pred[1])
    for i in range(1, num_params):
        for j in range(i + 1):
            if j == 0:
                H[i][j] *= x[i-1]
            else:
                H[i][j] *= x[i-1] * x[j-1] 
    i_lower = np.tril_indices(num_params, -1)
    H.T[i_lower] = H[i_lower]     
    return H

**First-order derivative of $P(y \mid \textbf{x})$ with respect to model parameters**

In [88]:
def del_f_del_theta_i(num_params, x, y_pred):
    del_f_del_theta = np.ones((num_params, 1)) * (y_pred[0] * y_pred[1])
    for j in range(1, num_params):
            del_f_del_theta[j] *=  x[j-1]
    return del_f_del_theta

**Computing $v=\nabla($Statistical parity difference$)$**

In [89]:
# Return v = del(SPD)/del(theta)
def del_spd_del_theta(num_params, X_test_orig, X_test, y_pred):
    del_f_protected = np.zeros((num_params, 1))
    del_f_privileged = np.zeros((num_params, 1))
    numProtected = X_test_orig['gender_Female'].sum()
    numPrivileged = X_test_orig['gender_Male'].sum()
    for i in range(len(X_test)):
        del_f_i = del_f_del_theta_i(num_params, X_test[i], y_pred[i])
        if X_test_orig.iloc[i]['gender_Male'] == 1: #privileged
            del_f_privileged = np.add(del_f_privileged, del_f_i)
        elif X_test_orig.iloc[i]['gender_Female'] == 1:
            del_f_protected = np.add(del_f_protected, del_f_i)
    del_f_privileged /= numPrivileged
    del_f_protected /= numProtected
    v = np.subtract(del_f_protected, del_f_privileged)
    return v

**Stochastic estimation of Hessian vector product (involving del fairness): $H_{\theta}^{-1}v = H_{\theta}^{-1}\nabla_{\theta}f(z, \theta) = v + [I - \nabla_{\theta}^2L(z_{s_j}, \theta^*)]H_{\theta}^{-1}v$**

In [90]:
# Uniformly sample t points from training data 
def hessian_vector_product(num_params, n, size, v, hessian_all_points):
    if (size > n):
        size = n
    sample = random.sample(range(n), size)
    hinv_v = copy.deepcopy(v)
    for idx in range(size):
        i = sample[idx]
        hessian_i = hessian_all_points[i]
        hinv_v = np.matmul(np.subtract(np.identity(num_params), hessian_i), hinv_v)
        hinv_v = np.add(hinv_v, v)
    return hinv_v

**First-order influence computation**

In [91]:
def first_order_influence(del_L_del_theta, hinv_v, n):
    infs = []
    for i in range(n):
        inf = -np.dot(del_L_del_theta[i].transpose(), hinv_v)
        inf *= -1/n
        infs.append(inf[0][0].tolist())
    return infs

**Second-order influence computation for a group of points in subset U**

In [97]:
def second_order_influence(X_train, U, size, del_L_del_theta, hessian_all_points):
    u = len(U)
    s = len(X_train)
    p = u/s
    c1 = (1 - 2*p)/(s * (1-p)**2)
    c2 = 1/((s * (1-p))**2)
    num_params = len(del_L_del_theta[0])
    del_L_del_theta_hinv = np.zeros((num_params, 1))
    del_L_del_theta_sum = np.zeros((num_params, 1))
    hessian_U = np.zeros((num_params, num_params))
    for i in range(u):
        idx = U[i]
        hessian_U = np.add(hessian_U, s * hessian_all_points[idx])
        del_L_del_theta_sum = np.add(del_L_del_theta_sum, del_L_del_theta[idx])
    
    hinv_del_L_del_theta= np.matmul(hinv_exact, del_L_del_theta_sum)
    hinv_hessian_U = np.matmul(hinv_exact, hessian_U)
    term1 = c1 * hinv_del_L_del_theta
    term2 = c2 * np.matmul(hinv_hessian_U, hinv_del_L_del_theta)
    sum_term = np.add(term1, term2)
    return sum_term

**Metrics: Initial state**

In [92]:
clf.fit(X_train, y_train)
num_params = len(clf.coef_.transpose()) + 1 #weights and intercept; params: clf.coef_, clf.intercept_
y_pred_test = clf.predict_proba(X_test)
y_pred_train = clf.predict_proba(X_train)
    
spd_0 = computeFairness(y_pred_test, X_test_orig)
print("Initial fairness: ", spd_0)

loss_0 = logistic_loss(y_test, y_pred_test)
print("Initial loss: ", loss_0)

accuracy_0 = computeAccuracy(y_test, y_pred_test)
print("Initial accuracy: ", accuracy_0)

Initial fairness:  -0.20059371090978573
Initial loss:  0.360972684923813
Initial accuracy:  0.7683939044369612


**Pre-compute: (1) Hessian (2) del_L_del_theta for each training data point**

In [93]:
del_L_del_theta = []
for i in range(int(len(X_train))):
    del_L_del_theta.insert(i, del_L_del_theta_i(num_params, y_train[i], X_train[i], y_pred_train[i]))

hessian_all_points = []
for i in range(len(X_train)):
    hessian_all_points.insert(i, hessian_one_point(num_params, X_train[i], y_pred_train[i])
                              /len(X_train))

*H^{-1} computation*

In [94]:
hexact = 1
v1 = del_spd_del_theta(num_params, X_test_orig, X_test, y_pred_test)
if hexact == 1: 
    H_exact = np.zeros((num_params, num_params))
    for i in range(len(X_train)):
        H_exact = np.add(H_exact, hessian_all_points[i])
    hinv_exact = np.linalg.pinv(H_exact) 
    hinv_v = np.matmul(hinv_exact, v1)
else: #using Hessian vector product
    size_hvp = int(len(X_train) * .01)
    hinv_v = hessian_vector_product(num_params, len(X_train), size_hvp, v1, hessian_all_points)

**Ground truth influence of each training data point**

In [95]:
# Ground truth influence
# spdgt = ground_truth_influence(X_train, y_train, X_test, X_test_orig)
# with open('delta_spd_ground_truth_v0.txt', 'w') as filehandle:
#     for listitem in delta_spd:
#         filehandle.write('%s\n' % listitem)
gt_spd = pd.read_csv('delta_spd_ground_truth_v0.txt', names=["Values"], sep=",")
gt_spd = gt_spd.values.tolist()
spdgt=[]
for i in range(len(gt_spd)):
    spdgt.append(gt_spd[i][0])

**First-order influence of each training data point**

In [96]:
infs_1 = first_order_influence(del_L_del_theta, hinv_v, len(X_train))

**Correlation between first-order and ground truth influences**

In [25]:
print("Spearman rank correlation between 1st order inf and ground truth inf: ", 
      stats.spearmanr(spdgt, infs_1)[0])
print("Pearson correlation coefficient between 1st order inf and ground truth inf: ", 
      stats.pearsonr(spdgt, infs_1)[0])

colors = (0.5,0.2,0.5)
fig = plt.figure()
plt.scatter(infs_1, spdgt, c=colors, alpha=0.5)
plt.xlabel('First-order influence')
plt.ylabel('Ground truth influence')
plt.show()

ValueError: all the input array dimensions for the concatenation axis must match exactly, but along dimension 0, the array at index 0 has size 30162 and the array at index 1 has size 500

**Verification of first-order influence (implementation) on log-loss of a single point**

In [31]:
def ground_truth_influence_loss(X_train, y_train, X_test, y_test, idx):
    clf.fit(X_train, y_train)
    y_pred = clf.predict_proba([X_test[idx]])[0]
    loss_0 = - y_test[idx] * math.log(y_pred[1]) - (1 - y_test[idx]) * math.log(y_pred[0])

    delta_loss = []
    for i in range(len(X_train)):
        X_removed = np.delete(X_train, i, 0)
        y_removed = y_train.drop(index=i, inplace=False)
        clf.fit(X_removed, y_removed)
        y_pred = clf.predict_proba([X_test[idx]])[0]
        loss_i = - y_test[idx] * math.log(y_pred[1]) - (1 - y_test[idx]) * math.log(y_pred[0])
        delta_loss_i = loss_i - loss_0
        delta_loss.append(delta_loss_i)
    
    return delta_loss

verify = 0
if verify:
    # Identify data points misclassified by the model
    clf.fit(X_train, y_train)
    y_pred = clf.predict_proba(X_test)
    for i in range(len(X_test)):
        if (y_pred[i][y_test[i]] < y_pred[i][1 - y_test[i]]):
            print(i)

    # compute loss approximated by first-order influence for a misclassified test data point
    ix = 203 
    v1 = del_L_del_theta_i(num_params, y_test[ix], X_test[ix], y_pred_test[ix])
    H_exact = np.zeros((num_params, num_params))
    for i in range(len(X_train)):
        H_exact = np.add(H_exact, hessian_all_points[i])
    hinv_exact = np.linalg.pinv(H_exact) 
    hinv_v = np.matmul(hinv_exact, v1)

    infs_1 = first_order_influence(del_L_del_theta, hinv_v, len(X_train))
    delta_loss_gt = ground_truth_influence_loss(X_train, y_train, X_test, y_test, ix)

    colors = (0.5,0.2,0.5)
    # fig = plt.figure()
    fig, ax = plt.subplots(figsize=(4,4))
    plt.scatter(infs_1, delta_loss_gt, c=colors, alpha=0.5)
    plt.xlabel('First-order influence')
    plt.ylabel('Ground truth influence')
    ax.plot((0,1), 'r--')
    # ax.plot([0,1],[0,1], transform=ax.transAxes)
    plt.show()

**Checking ground truth, first-order and second-order influences for a set**

In [73]:
predicates = ['marital_Never-married', 'gender_Female']
idx = X_train_orig[(X_train_orig[predicates[0]] == 1)
                   & (X_train_orig[predicates[1]] == 1) 
                  ].index 
print("#Rows removed: ", len(idx))
print("#Rows left: ", len(X_train) - len(idx))
X = np.delete(X_train, idx, 0)
y = y_train.drop(index=idx, inplace=False)
clf.fit(X, y)
y_pred_test = clf.predict_proba(X_test)
print("Ground truth fairness of subset: ", computeFairness(y_pred_test, X_test_orig))
print("Ground truth influence of subset: ", computeFairness(y_pred_test, X_test_orig) - spd_0)

del_f_1 = 0
for i in range(len(idx)):
    del_f_1 += infs_1[idx[i]]
print("First-order influence: ", del_f_1)

size_hvp = 1
params_f_2 = second_order_influence(X_train, idx, size_hvp, del_L_del_theta, hessian_all_points)
del_f_2 = np.dot(v1.transpose(), params_f_2)[0][0]
print("Second-order influence: ", del_f_2)



#Rows removed:  4312
#Rows left:  25850
Ground truth fairness of subset:  -0.20625215414793924
Ground truth influence of subset:  -0.00565844323815351
First-order influence:  -0.0022943232829864472
Second-order influence:  -0.004167267551007168


**Fairness: Ground-truth subset influence vs. computed subset influences: Random subset**

In [23]:
print("Ground-truth subset, Add 1st-order inf individual, Add ground-truth inf individual, Second-order subset influence")
sampleSize = int(.2 * len(X_train))
for i in range(100):
    idx = random.sample(range(1, len(X_train)), sampleSize) 
    X = np.delete(X_train, idx, 0)
    y = y_train.drop(index=idx, inplace=False)
    clf.fit(X, y)
    y_pred_test = clf.predict_proba(X_test)
    
    # Ground truth subset influence
    inf_gt = computeFairness(y_pred_test, X_test_orig) - spd_0
    
    # First-order subset influence
    del_f_1 = 0
    for i in range(len(idx)):
        del_f_1 += infs_1[idx[i]]
    
    # Second-order subset influence
    size_hvp = 1
    params_f_2 = second_order_influence(X_train, idx, size_hvp, del_L_del_theta, hessian_all_points)
    del_f_2 = np.dot(v1.transpose(), params_f_2)[0][0]
    
    print(inf_gt, del_f_1, del_f_gt, del_f_2, sep=", ")

Ground-truth subset, Add 1st-order inf individual, Add ground-truth inf individual, Second-order subset influence
0.0009629809259190658, 0.0007608505954095434, 0, 0.0009548758946811795
-0.0024558415138332956, -0.0019834973549170565, 0, -0.0024435206041296758
0.0007159796115257799, 0.0005817511282417933, 0, 0.0007062044545100855
0.0016977733073159085, 0.0012353292323012293, 0, 0.0016106707249222418
-0.0008390032699661376, -0.000794650035863751, 0, -0.0009214936138884197
0.0018752947121944419, 0.0015120618885418689, 0, 0.0018605853483757773
0.0047508936878143115, 0.003793923999265885, 0, 0.004636187042642229
-0.0008180093770101537, -0.000686133938715342, 0, -0.0009453824554443728
-0.00097319105268176, -0.0007660839715716599, 0, -0.0009792674820370327
0.0016469861163182498, 0.001342382859108319, 0, 0.001636853984906911
0.001218346289418748, 0.001030334564643214, 0, 0.0012184676544054083
0.001000833245967725, 0.0008488569873065311, 0, 0.0010078183911659024
-0.0004866763661370843, -0.000361

**Fairness: Ground-truth subset influence vs. computed subset influences: Coherent subset** 

(by coherent, we mean group of data points that share some properties)

In [101]:
print("Ground-truth subset, Add 1st-order inf individual, Add ground-truth inf individual, Second-order subset influence")
clf.fit(X_train, y_train)
continuous_cols = ['age', 'education.num', 'hours',]
# y_pred = clf.predict_proba([X_test[ix]])[0]
# loss_ix = - y_test[ix] * math.log(y_pred[1]) - (1 - y_test[ix]) * math.log(y_pred[0])
for col in X_train_orig.columns:
    vals = X_train_orig[col].unique()
    for val in vals:
#         print(col, val, sep=": ")
        if col in continuous_cols:
            idx = X_train_orig[X_train_orig[col] == val].index 
        else:
            idx = X_train_orig[X_train_orig[col] == 1].index 
        X = np.delete(X_train, idx, 0)
        y = y_train.drop(index=idx, inplace=False)
        if len(y.unique())>1:
            clf.fit(X, y)
            y_pred = clf.predict_proba(X_test)
#             y_pred = clf.predict_proba([X_test[ix]])[0]
#             inf_gt = - y_test[ix] * math.log(y_pred[1]) - (1 - y_test[ix]) * math.log(y_pred[0]) - loss_ix
            inf_gt = computeFairness(y_pred, X_test_orig)-spd_0
            del_f_gt = 0
            del_f_1 = 0
            diff = []
            for i in range(len(idx)):
                del_f_1 += infs_1[idx[i]]
#                 del_f_gt += delta_loss_gt[idx[i]] #spdgt[idx[i]]

            size_hvp = 1
            params_f_2 = second_order_influence(X_train, idx, size_hvp, del_L_del_theta, hessian_all_points)
            del_f_2 = np.dot(v1.transpose(), params_f_2)[0][0]

            print(inf_gt, del_f_1, del_f_gt, del_f_2, len(idx), sep=", ")

Ground-truth subset, Add 1st-order inf individual, Add ground-truth inf individual, Second-order subset influence
0.00032155796236454925, 0.00029454082652533996, 0, 0.00030328036997329463, 786
0.0011907157004082403, 0.0011543171079813003, 0, 0.001180300970831471, 575
2.7218782747251424e-05, 1.2118612942596616e-05, 0, -3.8425141131604313e-07, 791
0.0005263499141654471, 0.0005046670273034681, 0, 0.0005148722481624437, 448
-0.0008794362696055713, -0.0008384900270714601, 0, -0.0008687648463629457, 808
0.0012078108891511452, 0.0011554814772832803, 0, 0.0011815711584213942, 828
0.0005324645782582016, 0.0005128287129365049, 0, 0.0005208576370774723, 555
0.00024342051822168198, 0.000224524501465064, 0, 0.00023805720152062236, 455
-0.00038275424889328, -0.0003772696302561302, 0, -0.00038636324775190507, 851
-0.0001571038718774509, -0.00015012975792855852, 0, -0.00016694723767588833, 741
8.599345233262223e-05, 7.441341870919887e-05, 0, 8.297223274831478e-05, 813
-1.2616298432133188e-05, -1.59175

0.00020518398030416884, 0.0001806813285455724, 0, 0.0002022562587816609, 672
0.0005404065906345901, 0.0005208677799630199, 0, 0.000537101361893004, 494
6.3630520586333e-05, 5.801025060591234e-05, 0, 5.724292962436897e-05, 27
0.0001080208432066021, 0.00010304875849298792, 0, 0.00010336583355210776, 238
-0.0007161247343761001, -0.0006588411761220491, 0, -0.0007146510528770105, 276
-1.8217209975857074e-05, -2.2631751387643898e-05, 0, -2.2673887443399418e-05, 39
0.00012402612676287728, 0.00011659293601365963, 0, 0.00012048327562802879, 90
-0.00017297699364049457, -0.00017995746567534715, 0, -0.00017935087489683404, 34
-0.0001794915076853143, -0.00018073967282725714, 0, -0.00018065231137772054, 72
0.00015008072553598928, 0.00014764916561869242, 0, 0.00014634816451191012, 202
-0.0005155451534887001, -0.0005198843372466437, 0, -0.0005158652373725129, 217
-3.6919400352664056e-05, -4.22611711721743e-05, 0, -4.149630324203788e-05, 80
1.5124748653089215e-05, 9.827344460343383e-06, 0, 1.1228763763

0.0024088247367832216, 0.0006643311202665592, 0, 0.0007277911134415611, 943
0.0024088247367832216, 0.0006643311202665592, 0, 0.0007277911134415611, 943
0.0013416282098084875, 0.0001408496774750524, 0, 0.0009035685539643527, 2067
0.0013416282098084875, 0.0001408496774750524, 0, 0.0009035685539643527, 2067
0.016080198794457323, 0.002171886303439524, 0, 0.016496224471700162, 22286
0.016080198794457323, 0.002171886303439524, 0, 0.016496224471700162, 22286
0.002050567117793084, 0.0001404393948815077, 0, 0.00012375246478622148, 1074
0.002050567117793084, 0.0001404393948815077, 0, 0.00012375246478622148, 1074
-0.007413520413749708, -0.002113515182600834, 0, -0.002228453244490122, 2499
-0.007413520413749708, -0.002113515182600834, 0, -0.002228453244490122, 2499
-0.0017816691742466029, -0.0009986865136153975, 0, -0.0011110382977382476, 1279
-0.0017816691742466029, -0.0009986865136153975, 0, -0.0011110382977382476, 1279
-0.0002075749550743411, -2.417783930347705e-07, 0, -4.811153277243314e-07, 1

**Space Partitioner for reducing bias**

In [74]:
cols = ['age', 'workclass', 'fnlwgt', 'education', 'education.num', 'marital', 'occupation', 'relationship', 'race', 'gender', 'capgain', 'caploss', 'hours', 'country', 'income']
df_train = pd.read_csv('adult.data', names=cols, sep=",")
df_test = pd.read_csv('adult.test', names=cols, sep=",")

def preprocess(df):
    df.isin(['?']).sum(axis=0)

    # replace missing values (?) to nan and then drop the columns
    df['country'] = df['country'].replace('?',np.nan)
    df['workclass'] = df['workclass'].replace('?',np.nan)
    df['occupation'] = df['occupation'].replace('?',np.nan)

    # dropping the NaN rows now
    df.dropna(how='any',inplace=True)
    df['income'] = df['income'].map({'<=50K': 0, '>50K': 1}).astype(int)
    df = df.drop(columns=['fnlwgt', 'education.num', 'country', 'capgain', 'caploss'])
    return df

df_train = preprocess(df_train)
df_test = preprocess(df_test)

df_train = df_train.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

X_train_ = df_train.drop(columns='income')
y_train_ = df_train['income']

X_test_ = df_test.drop(columns='income')
y_test_ = df_test['income']

# size=100
# X_train = X_train[0:size]
# y_train = y_train[0:size]

In [75]:
def computeFairness(y_pred, X_test): 
    protected_idx = X_test[X_test['gender']=='Female'].index
    numProtected = len(protected_idx)
    privileged_idx = X_test[X_test['gender']=='Male'].index
    numPrivileged = len(privileged_idx)
    
    p_protected = 0
    for i in range(len(protected_idx)):
        p_protected += y_pred[protected_idx[i]][1]
    p_protected /= len(protected_idx)
    
    p_privileged = 0
    for i in range(len(privileged_idx)):
        p_privileged += y_pred[privileged_idx[i]][1]
    p_privileged /= len(privileged_idx)
    
    spd = p_protected - p_privileged
    return spd

def getInfluenceOfSet(indices, f, X_train, y_train, X_test, X_test_, method): 
    del_f = 0
    if (method == 1):
        X = X_train.drop(index=indices, inplace=False)
        y = y_train.drop(index=indices, inplace=False)
        if len(y.unique()) < 2:
            return 0
        clf.fit(X, y)
        y_pred = clf.predict_proba(X_test)
        del_f = computeFairness(y_pred, X_test_)
    elif (method == 2):
        for i in range(len(indices)):
            del_f += infs_1[indices[i]]
    elif (method == 3):
#         second_order_influence(X_train, U, size, del_L_del_theta, hessian_all_points)
        size_hvp = 1
        params_f_2 = second_order_influence(X_train, indices, size_hvp, del_L_del_theta, hessian_all_points)
        del_f = np.dot(v1.transpose(), params_f_2)[0][0]
    return  del_f

def getSplitVal(infs):
    return (np.argmax(np.asarray(infs)))

def getSplitAttribute(cols, cols_continuous, X_train, y_train, X_test, X_train_, X_test_, method):
    splitCol, numRows, score = None, len(X_train_), abs(spd_0)
    infs = []
    vals = []
    idxs = []
    for i in range(len(cols)):
        col = cols[i]
        infs_i = []
        vals_i = []
        idxs_i = []
        if col not in cols_continuous:
            colVals = X_train_[col].unique()
            for val in colVals:
                idx = X_train_[X_train_[col] == val].index
                idxs_i.append(len(X_train)-len(idx))
                infs_i.append(getInfluenceOfSet(idx, spd_0, X_train, y_train, X_test, X_test_, method))
                vals_i.append(val)
                ix = getSplitVal(infs_i)
                if (abs(infs_i[ix]) < score):
                    print("Column passed: ", col)
                    print("Val: ", val)
                    splitCol = col
                    splitIdx = i
                    score = abs(infs_i[ix])
        infs.append(infs_i)
        vals.append(vals_i)
        idxs.append(idxs_i)
    return {'splitCol':splitCol, 'numRows':numRows, 
            'infs': infs[splitIdx], 'vals': vals[splitIdx], 'idxs': idxs[splitIdx]}

def partition(node, maxDepth, minSize, depth, cols, cols_continuous, 
              X_train_, y_train_, X_train, X_test_, X_test, method):
    print("Depth: ", depth)
    if depth >= maxDepth or node['numRows'] < minSize:
        node['children'] = None
        return
    col = node['splitCol']
    if col not in cols_continuous:
        vals = X_train_[col].unique()
        child = [None] * len(vals)
        for i in range(len(vals)):
            idx = X_train_[X_train_[col] == vals[i]].index 
            X = X_train.drop(index=idx, inplace=False)
            y = y_train.drop(index=idx, inplace=False)
            X_ = X_train_.drop(index=idx, inplace=False)
            if len(X) < minSize:
                node['children'] = None
            else:
                cols_ = copy.deepcopy(cols)
                cols_.remove(col)
                child[i] = getSplitAttribute(cols_, cols_continuous,
                                             X, y, X_test, X_, X_test_, method)
                child[i]['col'] = col
                child[i]['val'] = vals[i]
                partition(child[i], maxDepth, minSize, depth + 1, cols_, cols_continuous, 
                  X_, y, X, X_test_, X_test, method)
    node['children'] = child

def buildTree(X_train_, X_train, maxDepth, minSize, method):
    cols = copy.deepcopy(X_train_.columns).tolist()
    cols_continuous = ['age', 'hours']
    X_train = pd.DataFrame(data=X_train, columns=X_train_orig.columns)
    cols = list(set(cols) - set(cols_continuous))
    root = getSplitAttribute(cols, cols_continuous,
                             X_train, y_train, X_test, X_train_, X_test_, method)
    print(root)
    partition(root, maxDepth, minSize, 1, cols, cols_continuous,
              X_train_, y_train_, X_train, X_test_, X_test, method)
    return root

method = 1
dtree = buildTree(X_train_, X_train, 2, 100, method)


Column passed:  relationship
Val:  Unmarried
Column passed:  race
Val:  White
Column passed:  workclass
Val:  Private
Column passed:  marital
Val:  Married-civ-spouse
{'splitCol': 'marital', 'numRows': 30162, 'infs': [-0.2083536018505245, -0.02827855385557211, -0.1890788115264716, -0.20065933554743223, -0.19875608458881688, -0.20070750164582, -0.19730181883177864], 'vals': ['Never-married', 'Married-civ-spouse', 'Divorced', 'Married-spouse-absent', 'Separated', 'Married-AF-spouse', 'Widowed'], 'idxs': [20436, 16097, 25948, 29792, 29223, 30141, 29335]}
Depth:  1
Column passed:  relationship
Val:  Husband
Column passed:  relationship
Val:  Unmarried
Column passed:  race
Val:  White
Column passed:  gender
Val:  Male
Depth:  2
Column passed:  relationship
Val:  Not-in-family
Column passed:  relationship
Val:  Unmarried
Column passed:  workclass
Val:  Federal-gov
Column passed:  occupation
Val:  Adm-clerical
Depth:  2
Column passed:  relationship
Val:  Not-in-family
Column passed:  relation

In [76]:
dtree

{'splitCol': 'marital',
 'numRows': 30162,
 'infs': [-0.2083536018505245,
  -0.02827855385557211,
  -0.1890788115264716,
  -0.20065933554743223,
  -0.19875608458881688,
  -0.20070750164582,
  -0.19730181883177864],
 'vals': ['Never-married',
  'Married-civ-spouse',
  'Divorced',
  'Married-spouse-absent',
  'Separated',
  'Married-AF-spouse',
  'Widowed'],
 'idxs': [20436, 16097, 25948, 29792, 29223, 30141, 29335],
 'children': [{'splitCol': 'gender',
   'numRows': 20436,
   'infs': [-0.05522396149784553, -0.09339174663761593],
   'vals': ['Male', 'Female'],
   'idxs': [5470, 14966],
   'col': 'marital',
   'val': 'Never-married',
   'children': None},
  {'splitCol': 'occupation',
   'numRows': 16097,
   'infs': [-0.0089975462644349,
    -0.02940095693213822,
    -0.04129455247150572,
    -0.01219578883237249,
    -0.020033867671982286,
    -0.013187659058612106,
    -0.029191105506546675,
    -0.024927399301536438,
    -0.026194908337570252,
    -0.02611891490945213,
    -0.0406203503